# Capítulo 6 - Estimação dos dados

## Bibliotecas básicas e outras inicializações

In [1]:
import warnings
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')

## Carregamento dos dados

In [2]:
dados = pd.read_excel('./datasets/Concrete_Data.xls', names=[
                      'cimento', 'escoria', 'cinzas', 'agua', 'plastificante', 'graudo', 'miudo', 'idade', 'resistencia'], 
                       na_values='?')
print(f'Linhas: {dados.shape[0]} | Colunas: {dados.shape[1]}')
dados.head()

Linhas: 1030 | Colunas: 9


,cimento,escoria,cinzas,agua,plastificante,graudo,miudo,idade,resistencia
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


## Dicionário dos dados

**Cimento**: quantidade em quilos para 1m³ de mistura. (float)

**Escória de alto forno**: quantidade em quilos para 1m³ de mistura. (float)

**Cinzas volantes**: quantidade em quilos para 1m³ de mistura. (float)

**Água**: quantidade em quilos para 1m³ de mistura. (float)

**Plastificante**: quantidade em quilos para 1m³ de mistura. (float)

**Agregado graudo**: quantidade em quilos para 1m³ de mistura. (float)

**Agregado miudo**: quantidade em quilos para 1m³ de mistura. (float)

**Idade**: em dias. (int)

**Resistência**: valor da resistência à compressão em megapascal (MPa). (float)

## Informação dos dados

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cimento        1030 non-null   float64
 1   escoria        1030 non-null   float64
 2   cinzas         1030 non-null   float64
 3   agua           1030 non-null   float64
 4   plastificante  1030 non-null   float64
 5   graudo         1030 non-null   float64
 6   miudo          1030 non-null   float64
 7   idade          1030 non-null   int64  
 8   resistencia    1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.5 KB


## Valores nulos

In [4]:
print('Verificando os valores nulos: \n')
for col in dados.columns:
    print('Coluna '+col+': '+str(dados[col].isnull().sum()))

Verificando os valores nulos: 

Coluna cimento: 0
Coluna escoria: 0
Coluna cinzas: 0
Coluna agua: 0
Coluna plastificante: 0
Coluna graudo: 0
Coluna miudo: 0
Coluna idade: 0
Coluna resistencia: 0


## Estatística dos dados

In [5]:
dados.describe()

,cimento,escoria,cinzas,agua,plastificante,graudo,miudo,idade,resistencia
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.165631,73.895485,54.187136,181.566359,6.203112,972.918592,773.578883,45.662136,35.817836
std,104.507142,86.279104,63.996469,21.355567,5.973492,77.753818,80.175427,63.169912,16.705679
min,102.000000,0.000000,0.000000,121.750000,0.000000,801.000000,594.000000,1.000000,2.331808
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.707115
50%,272.900000,22.000000,0.000000,185.000000,6.350000,968.000000,779.510000,28.000000,34.442774
75%,350.000000,142.950000,118.270000,192.000000,10.160000,1029.400000,824.000000,56.000000,46.136287
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.599225


## Separação dos dados

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dados.iloc[:, :8],
                                                    dados['resistencia'],
                                                    test_size=0.2,
                                                    random_state=265)

## Algoritmos de estimação

### Regressão Linear

In [7]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
_ = lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

Validação cruzada

In [8]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, dados.iloc[:, :8], dados['resistencia'], cv=5, scoring='r2', verbose=3, n_jobs=2)
print('R²: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


R²: 0.46 (+/- 0.18)


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    1.9s finished


Métricas

In [9]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

print(f'R²: {lr.score(X_test, y_test)}')
print('Erro Médio Absoluto:', mean_absolute_error(y_test, y_pred))
print('Erro Quadrático Médio:', mean_squared_error(y_test, y_pred))
print('Raiz do erro quadrático médio:', np.sqrt(mean_squared_error(y_test, y_pred)))

R²: 0.6165743526240688
Erro Médio Absoluto: 8.220392323793265
Erro Quadrático Médio: 105.7208501403803
Raiz do erro quadrático médio: 10.282064488242636


### Regressão polinomial

In [10]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

poly = Pipeline([
    ('clf', PolynomialFeatures(degree=3)),
    ('linear', LinearRegression(fit_intercept=False))
])

_ = poly.fit(X_train, y_train)
y_pred = poly.predict(X_test)

Validação cruzada

In [11]:
scores = cross_val_score(poly, dados.iloc[:, :8], dados['resistencia'], cv=5, scoring='r2', verbose=3, n_jobs=2)
print('R²: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


R²: -11.65 (+/- 29.32)


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    0.3s finished


Métricas

In [12]:
print(f'R²: {poly.score(X_test, y_test)}')
print('Erro Médio Absoluto:', mean_absolute_error(y_test, y_pred))
print('Erro Quadrático Médio:', mean_squared_error(y_test, y_pred))
print('Raiz do erro quadrático médio:', np.sqrt(mean_squared_error(y_test, y_pred)))

R²: 0.8229849943302822
Erro Médio Absoluto: 4.398153449131722
Erro Quadrático Médio: 48.80783801261584
Raiz do erro quadrático médio: 6.9862606602256


### Rede neural Perceptron

In [13]:
from sklearn.linear_model import Perceptron

perceptron = Perceptron()
_ = perceptron.fit(X_train, np.asarray(y_train, dtype='|S6'))
y_pred = perceptron.predict(X_test)

Validação cruzada

In [14]:
scores = cross_val_score(perceptron, dados.iloc[:, :8], np.asarray(
dados['resistencia'], dtype='|S6'), cv=5, scoring='r2', verbose=3, n_jobs=2)
print('R²: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


R²: -0.47 (+/- 0.91)


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    3.8s finished


Métricas

In [15]:
print('Erro Médio Absoluto:', mean_absolute_error(np.asarray(y_test, dtype='|S6'), y_pred))
print('Erro Quadrático Médio:', mean_squared_error(np.asarray(y_test, dtype='|S6'), y_pred))
print('Raiz do erro quadrático médio:', np.sqrt(mean_squared_error(np.asarray(y_test, dtype='|S6'), y_pred)))

Erro Médio Absoluto: 15.644968932038838
Erro Quadrático Médio: 374.3764791161165
Raiz do erro quadrático médio: 19.348810793330852


### MLP

In [16]:
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor()
_ = mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_test)

Validação cruzada

In [17]:
scores = cross_val_score(mlp, dados.iloc[:, :8], dados['resistencia'], cv=5, scoring='r2', verbose=3, n_jobs=2)
print('R²: %0.2f (+/- %0.2f)' % (scores.mean(), scores.std() * 2))

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


R²: 0.63 (+/- 0.28)


[Parallel(n_jobs=2)]: Done   5 out of   5 | elapsed:    4.9s finished


Métricas

In [18]:
print(f'R²: {mlp.score(X_test, y_test)}')
print('Erro Médio Absoluto:', mean_absolute_error(y_test, y_pred))
print('Erro Quadrático Médio:', mean_squared_error(y_test, y_pred))
print('Raiz do erro quadrático médio:', np.sqrt(mean_squared_error(y_test, y_pred)))

R²: 0.8240133368241763
Erro Médio Absoluto: 5.215524264721163
Erro Quadrático Médio: 48.524296096643326
Raiz do erro quadrático médio: 6.965938278268285
